In [2]:
import numpy as np
import json
from scipy.io.wavfile import read
import processing
import matplotlib.pyplot as plt
import json
import IPython.display as ipd
from scipy.signal import savgol_filter
import util
import quickPlayer

In [3]:
scores = np.load("scores-SFHA-MC.npy",allow_pickle = True)

In [4]:
soundDir = "musiccritic_cropped/"
annotationDir = soundDir + "Annotations/"
solofiles,chordfiles = util.separateFiles(soundDir,"mc")

In [6]:
i=110                      # some strumming examples 20,30,40,110
filename = scores[i][0]
print(filename)
filename in chordfiles

284_recording-0-2019-03-20T11-48-05-912Z-0


True

In [7]:
# load file and onset preds
filedir = annotationDir + filename
audio_filename = soundDir + filename+ ".wav"
fs, x = read(audio_filename)
x = x/np.max(np.abs(x))
gt = scores[i][2]*fs
pred = scores[i][3]*fs

# transform
M = [4095,2047]
N = 4096
H = 128
B = [[0,int(2000*N/fs)],[int(2000*N/fs),int(fs/2)]]
params= (M,N,H,B)
mX,pX = processing.fTransform(x,fs,params)

# sf, rms calculation and filtering
sf,sfn = processing.MRSF(mX,B=[0,int(2000*N/fs)],a=51,b=3,size=0.1)
rms = np.sum(mX,axis=1)
rms = rms-np.mean(rms)
rms = rms/np.max(rms)
rms = savgol_filter(rms,3,1)

In [8]:
# from the predicted onset locations, calculating chords' rise area using filtered rms, 
# predicting each string's onset inside the area

rise_areas = []
window = 100
strings = np.array([])
for o in pred:
    o = int(o/128)
    lw = np.min((window,o))
    rw = np.min((window,mX.shape[0]-o))
    rise_start = np.argmax(np.convolve(rms[o-lw+1:o+30]-rms[o-lw:o+29],np.ones(30))[30:])
    rise_end = np.argmax(np.convolve(rms[o+1:o+rw]-rms[o:o+rw-1],np.ones(30)*-1)[30:])
    rise_areas.append([o-lw+rise_start,o+rise_end])
    
    strings = np.append(strings,(o-lw+rise_start+processing.candidSelection(rms[o-lw+rise_start:o+rise_end+10],t=0.01,hw=3)))
    
quickPlayer.quickPlayer(audio_filename,[sf,rms],[gt,pred,strings*128],["gt","pred","string"],h=128,onlyValues=True,regions=rise_areas)

NameError: name 'stream' is not defined

TclError: invalid command name "."

<Figure size 1800x700 with 1 Axes>

In [10]:
help(quickPlayer.quickPlayer)

Help on function quickPlayer in module quickPlayer:

quickPlayer(soundfile, data, stamps, names, h=128, onlyValues=False, regions=None)
    onlyValues: If  provided time stamps does not have IDs or names (so just lines to be drawn), this should be True. Manually annotated files will have IDs and names for the stamps  . 
    
    soundfile: sound file to be played
    data: data that shown on the player plot
    stamps: lines will be drawed on these points on the data
    h: hopsize
    
    size(data) * h = size(sound)      If you pass audio samples as data, h (hop size) should be 1.



In [5]:
c = 0
bss = []
for onset in scores[70][6]:
    found = False
    octave_groups = []
    freqs = []
    be = []
    if onset!={}:
        for key in onset:

            for i in range(len(freqs)):
                oct_dist = np.log2(key)-np.log2(freqs[i][0])
                if np.abs(round(oct_dist)-oct_dist)<0.07:
                    if onset[key][0][1][0]<freqs[i][1][0]:
                        freqs[i][1][0]=onset[key][0][1][0]
                    if onset[key][0][1][1]>freqs[i][1][1]:
                        freqs[i][1][1]=onset[key][0][1][1]
                    found = True
                    break
                    
            if not found:
                freqs.append([key,onset[key][0][1]])
                
        bss.append(freqs)

In [145]:
# scores = np.load("scores-SFHA-GS_low.npy",allow_pickle = True)

# soundDir = "guitarset/"
# annotationDir = soundDir + "Annotations/"
# with open("guitarset/list_files.txt","r") as f:
#     filelist = json.load(f)
    
# chordfiles = []
# solofiles = []
# for i in range(360): # separating solo and accompaniment recording names
#     filename = filelist[i]
#     if filename.split("_")[2] == "comp":
#         chordfiles.append(filename)
#     else:
#         solofiles.append(filename)